This example just parses one web site and then uses it in langchain qa_chain chain_type = 'stuff'

In [9]:
from langchain.llms import OpenAI 
from dotenv import load_dotenv
from pathlib import Path
import os
dotenv_path = Path('.env_secure')
load_dotenv(dotenv_path=dotenv_path)
open_api_key = os.environ.get('OPENAI_API_KEY')

In [10]:

from bs4 import BeautifulSoup as bs

#https://www.aljazeera.com/news/2023/8/11/why-norways-record-breaking-climber-is-facing-backlash-after-sherpa-death

In [11]:
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("--headless=new")

from selenium import webdriver
driver = webdriver.Chrome(options=options)
url = 'https://www.theguardian.com/world/2023/aug/10/record-speed-mountaineer-denies-climbing-over-dying-sherpa-on-k2'
driver.get(url)
html = driver.page_source
soup = bs(html)
print(soup)

<html lang="en" style="top: 0px;"><head>
			    <!-- Hello there, HTML enthusiast! -->

				<!-- DCR commit hash 38c54354c32c6b0adcc8577ab35455b2d4d570d2 -->

                <title>Record-breaking mountaineer denies climbing over dying porter on K2 | Mountaineering | The Guardian</title>
                <meta content="Fellow climbers say video footage shows Kristin Harila’s team walking over body of frostbitten man during record ascent" name="description"/>
				<link href="https://www.theguardian.com/world/2023/aug/10/record-speed-mountaineer-denies-climbing-over-dying-sherpa-on-k2" rel="canonical"/>
                <meta charset="utf-8"/>
				<meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>
                <meta content="#052962" name="theme-color"/>
				<link href="https://assets.guim.co.uk/static/frontend/manifest.json" rel="manifest"/>
				<link href="https://assets.guim.co.uk/static/frontend/icons/homescreen/apple-touch-icon.svg" rel="apple-tou

In [12]:
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
all_text = ''
for tag in soup.find_all('p'):
    #print(tag.text)
    all_text += tag.text + '\n'
print(all_text)

Fellow climbers say video footage shows Kristin Harila’s team walking over body of frostbitten man during record ascent
A record-breaking mountaineer has denied allegations that her team climbed over a dying porter to reach the summit of K2 in Pakistan to become the world’s fastest climber to scale all peaks above 8,000 metres.
Kristin Harila climbed the world’s second highest mountain on 27 July along with her Nepali sherpa Tenjen (Lama) Sherpa, 35, to complete her 14th highest peak in just over three months to secure a new world record.
During the Norwegian’s ascent, porter Mohammed Hassan fell off a sheer edge at a height of about 8,200 metres. Harila, 37, has insisted her team did everything they could to save Hassan but conditions were too dangerous to move him.
Images have emerged of climbers clambering past Hassan on a ridge during Harila’s ascent.
Austrian climbing duo Wilhelm Steindl and Philip Flämig, who were also on K2 that day, said footage they later recorded using a dron

In [13]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
def get_text_chunks_langchain(text):
   text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
   docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
   return docs

In [14]:
doc1 = get_text_chunks_langchain(all_text)
print(doc1)
print(type(doc1))

[Document(page_content='Fellow climbers say video footage shows Kristin Harila’s team walking over body of frostbitten man during record ascent\nA record-breaking mountaineer has denied allegations that her team climbed over a dying porter to reach the summit of K2 in Pakistan to become the world’s fastest climber to scale all peaks above 8,000 metres.'), Document(page_content='Kristin Harila climbed the world’s second highest mountain on 27 July along with her Nepali sherpa Tenjen (Lama) Sherpa, 35, to complete her 14th highest peak in just over three months to secure a new world record.\nDuring the Norwegian’s ascent, porter Mohammed Hassan fell off a sheer edge at a height of about 8,200 metres. Harila, 37, has insisted her team did everything they could to save Hassan but conditions were too dangerous to move him.'), Document(page_content='Images have emerged of climbers clambering past Hassan on a ridge during Harila’s ascent.\nAustrian climbing duo Wilhelm Steindl and Philip Fläm

In [15]:
query = 'Was Kristin Harila accused of climbing over a porter.  What are the details?'
llm = OpenAI(temperature=0)
print(llm(query))



No, Kristin Harila was not accused of climbing over a porter. There are no details available about this incident.


In [17]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type='stuff')

response = chain.run(input_documents=doc1, question=query)
print(response)

 Yes, Kristin Harila was accused of climbing over a porter, Mohammed Hassan, during her record ascent of K2. Fellow climbers Wilhelm Steindl and Philip Flämig said footage they recorded using a drone showed climbers walking over his body instead of trying to rescue him. Harila has denied the allegations and said her team did everything they could to save Hassan.
